In [4]:
import pandas as pd

In [5]:

from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google'

In [7]:
df_age = pd.read_csv("/Users/diyavora/Desktop/merging/ages.csv")
df_gdp=pd.read_csv("/Users/diyavora/Desktop/merging/gdp.csv",on_bad_lines='skip')
df_schools=pd.read_csv("/Users/diyavora/Desktop/merging/schools.csv")

In [8]:
#checking if they are loaded
df_age.head()

,Country,HS_age_population,HS_age_population_billion,HS_age_population_pct
0,China,1.377710e+10,13.777,19.307
1,India,1.286782e+10,12.868,18.033
2,United States of America,2.770717e+09,2.771,3.883
3,Indonesia,2.539521e+09,2.540,3.559
4,Brazil,2.002351e+09,2.002,2.806


In [9]:
df_gdp.head()


,Country Name,GDP per capita (PPP)
0,Africa Eastern and Southern,4481.246847
1,Africa Western and Central,5639.830732
2,Albania,23488.098947
3,Algeria,17552.774248
4,Andorra,74939.474604


In [10]:
df_schools.head()

,School Name,City,Country,URL
0,Good Hope Country Day School,Frederiksted,-,NaN
1,International Learning Group School (ILG School),Pristina,-,NaN
2,Robinson School,San Juan,-,NaN
3,Two Boats School,Ascension,-,NaN
4,Virgin Islands Montessori School & Peter Grube...,St. Thomas,-,NaN


In [11]:
print("Ages dataset columns:", df_age.columns.tolist())
print("GDP dataset columns:", df_gdp.columns.tolist())
print("Schools dataset columns:", df_schools.columns.tolist())

print("\nSample rows from each:")
print(df_age.head(2))
print(df_gdp.head(2))
print(df_schools.head(2))

Ages dataset columns: ['Country', 'HS_age_population', 'HS_age_population_billion', 'HS_age_population_pct']
GDP dataset columns: ['Country Name', 'GDP per capita (PPP)']
Schools dataset columns: ['School Name', 'City', 'Country', 'URL']

Sample rows from each:
  Country  HS_age_population  HS_age_population_billion  HS_age_population_pct
0   China       1.377710e+10                     13.777                 19.307
1   India       1.286782e+10                     12.868                 18.033
                  Country Name  GDP per capita (PPP)
0  Africa Eastern and Southern           4481.246847
1   Africa Western and Central           5639.830732
                                        School Name          City Country  URL
0                      Good Hope Country Day School  Frederiksted       -  NaN
1  International Learning Group School (ILG School)      Pristina       -  NaN


In [12]:
school_counts = df_schools.groupby('Country')['School Name'].count().reset_index(name='Num_Schools')
school_counts

,Country,Num_Schools
0,-,5
1,Albania,4
2,Algeria,1
3,Angola,3
4,Anguilla,1
...,...,...
158,Venezuela,9
159,Vietnam,63
160,Yemen,1
161,Zambia,9


In [13]:
# Standardize column names for merging
df_gdp = df_gdp.rename(columns={'Country Name': 'Country'})
df_age = df_age.rename(columns={'HS_age_population': 'Youth_Population'})

In [14]:
# Merge all three datasets
merged = (
    school_counts
    .merge(df_gdp[['Country', 'GDP per capita (PPP)']], on='Country', how='left')
    .merge(df_age[['Country', 'Youth_Population']], on='Country', how='left')
)

In [15]:
merged.head()
merged = merged.iloc[1:]

In [16]:
merged

,Country,Num_Schools,GDP per capita (PPP),Youth_Population
1,Albania,4,23488.098947,37097000.0
2,Algeria,1,17552.774248,372104000.0
3,Angola,3,8347.952791,222522000.0
4,Anguilla,1,28561.300000,145000.0
5,Argentina,36,30175.535966,405125000.0
...,...,...,...,...
158,Venezuela,9,8404.000000,287449000.0
159,Vietnam,63,16385.506557,894347000.0
160,Yemen,1,1996.000000,272163000.0
161,Zambia,9,4223.950208,150325000.0


In [17]:
merged.shape

(162, 4)

In [21]:
merged['GDP_scaled'] = (merged['GDP per capita (PPP)'] - merged['GDP per capita (PPP)'].min()) / \
                       (merged['GDP per capita (PPP)'].max() - merged['GDP per capita (PPP)'].min())

merged['Schools_scaled'] = (merged['Num_Schools'] - merged['Num_Schools'].min()) / \
                           (merged['Num_Schools'].max() - merged['Num_Schools'].min())

merged['Youth_scaled'] = (merged['Youth_Population'] - merged['Youth_Population'].min()) / \
                         (merged['Youth_Population'].max() - merged['Youth_Population'].min())

# Invert youth: smaller youth population = higher per-capita opportunity
merged['Youth_scaled_inverted'] = 1 - merged['Youth_scaled']

merged['F1_scaled'] = (merged['F1_Visas'] - merged['F1_Visas'].min()) / \
                      (merged['F1_Visas'].max() - merged['F1_Visas'].min())


KeyError: 'Youth Population'